In [1]:
import ROOT
from analysis_framework import Dataset, Analysis

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8316680


In [2]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
plot_dir_postfix = "-new-cuts"
# dataset_path = "data/datasets/miniDSTs/processed-test.json"
dataset_path = "data/datasets/miniDSTs/processed-test-smaller.json"
output_path = "root://eosproject.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/test"
output_meta = "test.json"
checked_output_meta = "checked-test.json"
output_collections = r"^(?!R2jet_lvecs)\w+$" # ["PandoraPFOs"]
plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # output_path = "root://eosproject.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/full"
    # output_meta = "full.json"
    checked_output_meta = "checked-full.json"
    output_collections = r"^(?!R2jet_lvecs)\w+$"
    # plot_dir = "plots/pre-selection/full"
    plot_dir = f"plots/pre-selection/min-aa-min-higgs{plot_dir_postfix}"


In [3]:
ROOT.EnableImplicitMT(n_threads)

In [4]:
dataset = Dataset.from_json(dataset_path)

In [5]:
analysis = Analysis(dataset)

analysis.init_parameters([
    ("WWCategorisation.RecoCatBasic", "int", "8"),
    ("WWCategorisation.RecoCatAdvanced", "int", "8"),
    ("WWCategorisation.TrueCat", "int", "8"),
    ("WWCategorisation.missE", "float", "-42"),
    ("WWCategorisation.misspT", "float", "-42"),
    ("WWCategorisation.mInv", "float", "-42"),
    ("Energy", "float", "-42"),
])

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xc724690


Warning in <TClass::Init>: no dictionary for class edm4hep::Vector2i is available


In [6]:
# signal cut goes here:
# In this case, we want to require the charged lepton in the final state to be inside of the detector acceptance
# just take the first gen status 1 particle with a fitting pdg value and cut on theta
lep_pdg = 11
# CLD 150 mrad
# ILD ~84 mrad (IDR: 4.8 deg)
acceptance_theta = 0.084
# signal_cut = f"""
# std::invoke([](auto& genStat, auto& pdg, auto& px, auto& py, auto& pz, auto& m) -> bool {{
# auto lepton_mask = genStat == 1 && abs(pdg) == {lep_pdg};
# // abuse ArgMax to get the first set position
# auto lepton_idx = ArgMax(lepton_mask);
# auto lepton_lvec = ROOT::Math::PxPyPzMVector(px[lepton_idx], py[lepton_idx],
#                                              pz[lepton_idx], m[lepton_idx]);
# double lepton_theta = lepton_lvec.Theta();
# return abs(cos(lepton_theta)) < cos({acceptance_theta});
# }}, MCParticlesSkimmed.generatorStatus, MCParticlesSkimmed.PDG, MCParticlesSkimmed.momentum.x, MCParticlesSkimmed.momentum.y, MCParticlesSkimmed.momentum.z, MCParticlesSkimmed.mass)
# """
signal_cut = "params_WWCategorisation_TrueCat == 2"

In [7]:
categories = {
    "4f_sw_sl_signal": {"pattern": "4f_sw_sl", "cut": signal_cut},
    "4f_sl_bkg": {"pattern": r"4f\w+sl", "cut": None }, # inverse signal cut will be applied automatically
    "4f_not_sl": {"pattern": r"4f\w+_(?:h|l)", "cut": None },
    # separate out aa_4f? super low lumi anyway
    "aa2f": {"pattern": "aa_2f", "cut": None},
    # 2f but not aa_2f
    "2f": {"pattern": "(?<!aa_)2f", "cut": None},
    "3f": {"pattern": "ea_3f|ae_3f", "cut": None},
    "5f": {"pattern": "ea_5f|ae_5f", "cut": None},
    "6f": {"pattern": "6f", "cut": None},
    # need to filter out anything ending in _h and 2f_z_eehiq
    "higgs": {"pattern": "[^_e]h", "cut": None},
}
analysis.set_categories(categories)
# check if we missed any processes
print(analysis.is_complete_categorisation())

True


In [8]:
# needed for the .size() calls... alternative would probably be to .Alias the @size columns
ROOT.gInterpreter.Declare("#include <podio/ObjectID.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/ReconstructedParticleData.h>")
ROOT.gInterpreter.Declare("#include \"analyzers.h\"")
analysis.Define("n_isomuons", "IsolatedMuons_objIdx.size()")
analysis.Define("n_isoelectrons", "IsolatedElectrons_objIdx.size()")
analysis.Define("n_isophotons", "IsolatedPhotons_objIdx.size()")
analysis.Define("n_isotaus", "IsolatedTaus.size()")
analysis.Define("RecoCatBasic", "params_WWCategorisation_RecoCatBasic")
analysis.Define("RecoCatAdvanced", "params_WWCategorisation_RecoCatAdvanced")
analysis.Define("n_charged_PFOs", "Sum(abs(PandoraPFOs.charge) == 1.)")
analysis.Define("n_R2Jets", "Refined2Jets.size()")
make_lvec = lambda coll_name: f"ROOT::VecOps::Construct<ROOT::Math::PxPyPzEVector>({coll_name}.momentum.x, {coll_name}.momentum.y, {coll_name}.momentum.z, {coll_name}.energy)"
analysis.Define("R2Jet_n_pfos", "Refined2Jets.particles_end - Refined2Jets.particles_begin")
analysis.Define("R2Jet_lvecs", make_lvec("Refined2Jets"))
analysis.Define("R2Jet_theta", "return Map(R2Jet_lvecs, [] (const auto& el) {return el.Theta();} )")
analysis.Define("R2Jet_abscostheta", "-abs(cos(R2Jet_theta))")
analysis.Define("M_jj", "ROOT::VecOps::Sum(R2Jet_lvecs, ROOT::Math::PxPyPzEVector()).mass()")
analysis.Define("PVertex_ndf", "PrimaryVertex.ndf")
analysis.Define("PVertex_chi2", "PrimaryVertex.chi2")
analysis.Define("PVertex_chi2ndf", "PVertex_chi2 / PVertex_ndf")
analysis.Define("n_photons", "Sum(PandoraPFOs.PDG == 22)")
analysis.Define("n_neutrals", "Sum(abs(PandoraPFOs.PDG) == 2112)")
analysis.Define("PFO_lvecs", make_lvec("PandoraPFOs"))
analysis.Define("PFO_theta", "return Map(PFO_lvecs, [] (const auto& el) {return el.Theta();})")
analysis.Define("PFO_pt", "return Map(PFO_lvecs, [] (const auto& el) {return el.Pt();})")
analysis.Define("PFO_ovl_idx", "log10(PFO_pt) <= -2*sin(PFO_theta)+0.2")
analysis.Define("PFO_not_ovl_idx", "log10(PFO_pt) > -2*sin(PFO_theta)+0.2")
analysis.Define("n_not_ovl_PFOs", "Sum(PFO_not_ovl_idx)")
analysis.Define("clean_R2Jets", "remove_constituents(Refined2Jets, _Refined2Jets_particles, PandoraPFOs, PFO_ovl_idx)")
analysis.Define("clean_R2Jet_lvecs", "return Map(clean_R2Jets, [] (const auto& el) {return ROOT::Math::PxPyPzEVector(el.momentum.x, el.momentum.y, el.momentum.z, el.energy);})")
analysis.Define("clean_M_jj", "ROOT::VecOps::Sum(clean_R2Jet_lvecs, ROOT::Math::PxPyPzEVector()).mass()")
analysis.Define("clean_R2Jet_theta", "return Map(clean_R2Jet_lvecs, [] (const auto& el) {return el.Theta();} )")
analysis.Define("clean_R2Jet_abscostheta", "-abs(cos(clean_R2Jet_theta))")
analysis.Define("BCal_E", "Sum(BCalRecoParticle.energy)")
analysis.Define("n_BCal", "BCalRecoParticle.size()")
analysis.Define("isoelectron_lvecs", "PFO_lvecs[subset_to_mask(PandoraPFOs, IsolatedElectrons_objIdx)]")
analysis.Define("isoelectron_pt", "return Map(isoelectron_lvecs, [] (const auto& el) {return el.Pt();})")
analysis.Define("isoelectron_leading_pt", "std::max(Max(isoelectron_pt), 0.)")
analysis.Define("isoelectron_E", "return Map(isoelectron_lvecs, [] (const auto& el) {return el.energy();})")
analysis.Define("isoelectron_leading_E", "std::max(Max(isoelectron_E), 0.)")
analysis.Define("isoelectron_P", "return Map(isoelectron_lvecs, [] (const auto& el) {return el.P();})")
analysis.Define("isoelectron_Psum", "Sum(isoelectron_P)")
analysis.Define("isomuon_lvecs", "PFO_lvecs[subset_to_mask(PandoraPFOs, IsolatedMuons_objIdx)]")
analysis.Define("isomuon_pt", "return Map(isomuon_lvecs, [] (const auto& el) {return el.Pt();})")
analysis.Define("isomuon_leading_pt", "std::max(Max(isomuon_pt), 0.)")
analysis.Define("isomuon_E", "return Map(isomuon_lvecs, [] (const auto& el) {return el.energy();})")
analysis.Define("isomuon_leading_E", "std::max(Max(isomuon_E), 0.)")
analysis.Define("isomuon_P", "return Map(isomuon_lvecs, [] (const auto& el) {return el.P();})")
analysis.Define("isomuon_Psum", "Sum(isomuon_P)")
analysis.Define("isotaus_lvecs", make_lvec("IsolatedTaus"))
analysis.Define("isotaus_pt", "return Map(isotaus_lvecs, [] (const auto& el) {return el.Pt();})")
analysis.Define("isotaus_leading_pt", "std::max(Max(isotaus_pt), 0.)")
analysis.Define("isotaus_E", "return Map(isotaus_lvecs, [] (const auto& el) {return el.energy();})")
analysis.Define("isotaus_leading_E", "std::max(Max(isotaus_E), 0.)")
analysis.Define("isotaus_P", "return Map(isotaus_lvecs, [] (const auto& el) {return el.P();})")
analysis.Define("isotaus_Psum", "Sum(isotaus_P)")
analysis.Define("leading_iso_Psum", "ArgMax(ROOT::RVecD({isoelectron_Psum, isomuon_Psum, isotaus_Psum}))")
analysis.Define("clean_E_sum", "Sum(PandoraPFOs.energy[PFO_not_ovl_idx])")

signal_category = ["4f_sw_sl_signal"]
# analysis.define_truth_objects(signal_category)
# analysis.define_reco_objects(x_angle)
# analysis.remove_x_angle(x_angle)
# analysis.Define("M_Wlep", "ub_leptonic_W_lvec.M()")
# analysis.Define("M2_Wlep_sign", "M_Wlep / abs(M_Wlep)")

In [9]:
analysis.book_histogram_1D("RecoCatBasic", "RecoCatBasic", ("", ";RecoCatBasic", 8, 0., 8.))
# analysis.book_histogram_1D("RecoCatAdvanced", "RecoCatAdvanced", ("", ";RecoCatAdvanced", 8, 0., 8.))
# analysis.add_filter("RecoCatAdvanced == 2", "RecoCatAdvanced == 2")

In [10]:
analysis.book_histogram_1D("n_isoelectrons", "n_isoelectrons", ("", ";n_isoelectrons", 5, 0., 5.))
analysis.add_filter("n_isoelectrons == 1", "1 iso e^{#pm}")

analysis.book_histogram_1D("leading_iso_Psum", "leading_iso_Psum", ("", ";leading_iso_Psum", 3, 0., 3.))
analysis.add_filter("leading_iso_Psum == 0", "leading iso = e^{#pm}")

analysis.book_histogram_1D("n_charged_PFOs", "n_charged_PFOs", ("", ";n_charged_PFOs", 50, 0., 50.))
analysis.add_filter("n_charged_PFOs >= 10", "N_{#pm}#geq 10")

analysis.book_histogram_1D("n_not_ovl_PFOs", "n_not_ovl_PFOs", ("", ";n_not_ovl_PFOs", 125, 0., 125.))
analysis.add_filter("n_not_ovl_PFOs >= 20", "N_{clean} #geq 20")
analysis.add_filter("n_not_ovl_PFOs <= 100", "N_{clean} #leq 100")

analysis.book_histogram_1D("mInv", "params_WWCategorisation_mInv", ("", ";mInv", 250, 0., 250.))
analysis.add_filter("params_WWCategorisation_mInv >= 40", "M_{jj} #geq 40 GeV")
analysis.add_filter("params_WWCategorisation_mInv <= 140", "M_{jj} #leq 140 GeV")

# analysis.book_histogram_1D("RecoCatAdvanced", "RecoCatAdvanced", ("", ";RecoCatAdvanced", 8, 0., 8.))
# analysis.book_histogram_1D("RecoCatBasic", "RecoCatBasic", ("", ";RecoCatBasic", 8, 0., 8.))
# fully contained in the other cuts now!
# analysis.add_filter("RecoCatAdvanced == 2", "RecoCat = 2")

analysis.book_histogram_1D("misspT", "params_WWCategorisation_misspT", ("", ";misspT", 100, 0., 100.))
analysis.add_filter("params_WWCategorisation_misspT >= 10", "misspT#geq 10")

# analysis.book_histogram_1D("M2_Wlep_sign", "M2_Wlep_sign", ("", ";M2_Wlep_sign", 5, -1.5, 3.5))
# analysis.add_filter("M2_Wlep_sign > 0", "M2_Wlep_sign > 0")

analysis.book_histogram_1D("PVertex_ndf", "PVertex_ndf", ("", ";PVertex_ndf", 50, 1., 101.))
analysis.book_histogram_1D("missE", "params_WWCategorisation_missE", ("", ";missE", 300, -50., 250.))
analysis.book_histogram_1D("M_jj", "M_jj", ("", ";M_jj", 100, 32., 132.))
analysis.book_histogram_1D("clean_M_jj", "clean_M_jj", ("", ";clean_M_jj", 100, 32., 132.))
analysis.book_histogram_1D("M_jj_zoom", "M_jj", ("", ";M_jj", 100, 70., 100.))
analysis.book_histogram_1D("n_isomuons", "n_isomuons", ("", ";n_isomuons", 10, 0., 10.))
analysis.book_histogram_1D("n_isotaus", "n_isotaus", ("", ";n_isotaus", 10, 0., 10.))
analysis.book_histogram_1D("n_isophotons", "n_isophotons", ("", ";n_isophotons", 10, 0., 10.))
analysis.book_histogram_1D("n_BCal", "n_BCal", ("", ";n_BCal", 10, 0., 10.))
analysis.book_histogram_1D("n_photons", "n_photons", ("", ";n_photons", 45, 0., 45.))
analysis.book_histogram_1D("n_neutrals", "n_neutrals", ("", ";n_neutrals", 45, 0., 45.))
analysis.book_histogram_1D("PVertex_chi2", "PVertex_chi2", ("", ";PVertex_chi2", 75, 0., 150.))
analysis.book_histogram_1D("PVertex_chi2ndf", "PVertex_chi2ndf", ("", ";PVertex_chi2/ndf", 20, 0., 10.))
analysis.book_histogram_1D("R2Jet_abscostheta", "R2Jet_abscostheta", ("", ";- R2Jet_abscostheta", 75, -1., 0.))
analysis.book_histogram_1D("R2Jet_n_pfos", "R2Jet_n_pfos", ("", ";R2Jet_n_pfos", 50, 0., 50.))
analysis.book_histogram_1D("clean_R2Jet_abscostheta", "clean_R2Jet_abscostheta", ("", ";- clean_R2Jet_abscostheta", 75, -1., 0.))
analysis.book_histogram_1D("BCal_E", "BCal_E", ("", ";BCal_E", 125, 0., 125.))
analysis.book_histogram_1D("isoelectron_leading_pt", "isoelectron_leading_pt", ("", ";isoelectron_leading_pt", 125, 0., 125.))
analysis.book_histogram_1D("isoelectron_leading_E", "isoelectron_leading_E", ("", ";isoelectron_leading_E", 125, 0., 125.))
analysis.book_histogram_1D("isoelectron_Psum", "isoelectron_Psum", ("", ";isoelectron_Psum", 250, 0., 250.))
analysis.book_histogram_1D("isomuon_leading_pt", "isomuon_leading_pt", ("", ";isomuon_leading_pt", 125, 0., 125.))
analysis.book_histogram_1D("isomuon_leading_E", "isomuon_leading_E", ("", ";isomuon_leading_E", 125, 0., 125.))
analysis.book_histogram_1D("isomuon_Psum", "isomuon_Psum", ("", ";isomuon_Psum", 250, 0., 250.))
analysis.book_histogram_1D("clean_E_sum", "clean_E_sum", ("", ";clean_E_sum", 300, 0., 300.))

In [11]:
analysis.book_reports()

In [12]:
# analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [13]:
%%time
analysis.run()

CPU times: user 9min 28s, sys: 1min 41s, total: 11min 10s
Wall time: 11min 24s


In [14]:
# analysis.check_snapshots("events", output_path, checked_output_meta)

In [15]:
analysis.print_reports()
analysis.draw_cutflow(plot_dir=plot_dir)

       4f_sw_sl_signal             4f_sl_bkg             4f_not_sl                  aa2f                    2f                    3f                    5f                    6f                 higgs
        12987547 (1e+04)        33765152 (7e+04)        75996241 (8e+04)      7613111686 (2e+07)      1272531834 (1e+06)       280227320 (3e+05)          132015 (6e+02)            2918 (1e+01)         1050064 (7e+03) All
        11434727 (1e+04)         3098143 (1e+04)        13157604 (2e+04)       468283809 (4e+06)       404764002 (6e+05)        49931234 (1e+05)           32447 (3e+02)             534 (5e+00)           57337 (2e+03) 1 iso e^{#pm}
        11264658 (1e+04)         2857504 (1e+04)        10253713 (1e+04)       445382825 (4e+06)       329174683 (5e+05)        43506922 (1e+05)           23733 (2e+02)             450 (4e+00)           43848 (1e+03) leading iso = e^{#pm}
        11229680 (1e+04)         2790596 (1e+04)         3234061 (9e+03)        94072412 (2e+06)        913735

Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('cut flow', 5000, 0.0, 0.0).pdf has been created


In [16]:
# analysis.post_selection_stats("2f", dir=plot_dir)
# for category_name in categories:
    # analysis.post_selection_stats(category_name, dir=plot_dir)

In [17]:
analysis.draw_histogram("n_isoelectrons", logY=True, plot_dir=plot_dir, x_arrowl=1.0, x_arrowr=2.0, overlay=signal_category)
analysis.draw_histogram("leading_iso_Psum", logY=True, plot_dir=plot_dir, x_arrowl=0., x_arrowr=1.0, overlay=signal_category)
analysis.draw_histogram("n_charged_PFOs", logY=True, plot_dir=plot_dir, x_arrowl=10., overlay=signal_category)
analysis.draw_histogram("n_not_ovl_PFOs", logY=True, plot_dir=plot_dir, x_arrowl=20., x_arrowr=100., overlay=signal_category)
analysis.draw_histogram("mInv", logY=True, plot_dir=plot_dir, x_arrowl=40.0, x_arrowr=140.0, overlay=signal_category)
# analysis.draw_histogram("RecoCatAdvanced", logY=True, plot_dir=plot_dir, x_arrowl=2., x_arrowr=3., overlay=signal_category)

analysis.draw_histogram("misspT", logY=True, plot_dir=plot_dir, x_arrowl=10., overlay=signal_category)
# analysis.draw_histogram("M2_Wlep_sign", plot_dir=plot_dir, x_arrowl=0.5)

(<cppyy.gbl.THStack object at 0x4fab9860>,
 <cppyy.gbl.TCanvas object at 0x5307fb10>)

Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('n_isoelectrons', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('leading_iso_Psum', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('n_charged_PFOs', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('n_not_ovl_PFOs', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('mInv', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('misspT', 5000, 0.0, 0.0).pdf has been created


In [18]:
plot_logY = False

analysis.draw_histogram("PVertex_ndf", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("missE", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("M_jj", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("clean_M_jj", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("M_jj_zoom", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_isomuons", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_isotaus", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_isophotons", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_photons", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_neutrals", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("n_BCal", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("R2Jet_abscostheta", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("clean_R2Jet_abscostheta", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("R2Jet_n_pfos", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("BCal_E", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isoelectron_leading_pt", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isoelectron_leading_E", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isoelectron_Psum", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isomuon_leading_pt", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isomuon_leading_E", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("isomuon_Psum", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("clean_E_sum", logY=plot_logY, plot_dir=plot_dir, overlay=signal_category)

(<cppyy.gbl.THStack object at 0x499706c0>,
 <cppyy.gbl.TCanvas object at 0x4ee70e60>)

Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('PVertex_ndf', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('missE', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('M_jj', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('clean_M_jj', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('M_jj_zoom', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('n_isomuons', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('n_isotaus', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/

In [19]:
analysis.draw_histogram("PVertex_chi2", logY=True, plot_dir=plot_dir, overlay=signal_category)
analysis.draw_histogram("PVertex_chi2ndf", logY=True, plot_dir=plot_dir, overlay=signal_category)

(<cppyy.gbl.THStack object at 0x50e0ef00>,
 <cppyy.gbl.TCanvas object at 0x50ceefe0>)

Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('PVertex_chi2', 5000, 0.0, 0.0).pdf has been created
Info in <TCanvas::Print>: pdf file plots/pre-selection/min-aa-min-higgs-new-cuts/('PVertex_chi2ndf', 5000, 0.0, 0.0).pdf has been created
